![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Train POS Tagger in French by Spark NLP
### Based on Universal Dependency `UD_French-GSD`


In [ ]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.4.0

In [1]:
import sys
import time

#Spark ML and SQL
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
#Spark NLP
import sparknlp
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher


### Let's create a Spark Session for our app

In [2]:
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.0
Apache Spark version:  2.4.4


Let's prepare our training datasets containing `token_posTag` like `de_DET`. You can download this data set from Amazon S3:

```
wget -N https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/fr/pos/UD_French/UD_French-GSD_2.3.txt -P /tmp
```

In [3]:
! wget -N https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/fr/pos/UD_French/UD_French-GSD_2.3.txt -P /tmp

--2020-02-10 19:39:50--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/fr/pos/UD_French/UD_French-GSD_2.3.txt
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.133.205
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.133.205|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/tmp/UD_French-GSD_2.3.txt’ not modified on server. Omitting download.



In [4]:
from sparknlp.training import POS
training_data = POS().readDataset(spark, '/tmp/UD_French-GSD_2.3.txt', '_', 'tags')

In [5]:
training_data.show()

+--------------------+--------------------+--------------------+
|                text|            document|                tags|
+--------------------+--------------------+--------------------+
|Les commotions cé...|[[document, 0, 11...|[[pos, 0, 2, DET,...|
|L' œuvre est situ...|[[document, 0, 82...|[[pos, 0, 1, DET,...|
|Le comportement d...|[[document, 0, 18...|[[pos, 0, 1, DET,...|
|Toutefois , les f...|[[document, 0, 44...|[[pos, 0, 8, ADV,...|
|Ismene entre et a...|[[document, 0, 80...|[[pos, 0, 5, PROP...|
|je reviendrais av...|[[document, 0, 28...|[[pos, 0, 1, PRON...|
|Les forfaits comp...|[[document, 0, 30...|[[pos, 0, 2, DET,...|
|Il prévient que d...|[[document, 0, 99...|[[pos, 0, 1, PRON...|
|Ils tiraient à ba...|[[document, 0, 43...|[[pos, 0, 2, PRON...|
|Le château est en...|[[document, 0, 44...|[[pos, 0, 1, DET,...|
|En effet , la bir...|[[document, 0, 10...|[[pos, 0, 1, ADP,...|
|Le point final de...|[[document, 0, 15...|[[pos, 0, 1, DET,...|
|L' information gé...|[[d

In [6]:
document_assembler = DocumentAssembler() \
    .setInputCol("text")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")\
    .setExceptions(["jusqu'", "aujourd'hui", "États-Unis", "lui-même", "celui-ci", "c'est-à-dire", "celle-ci", "au-dessus", "etc.", "sud-est", "Royaume-Uni", "ceux-ci", "au-delà", "elle-même", "peut-être", "sud-ouest", "nord-ouest", "nord-est", "Etats-Unis", "Grande-Bretagne", "Pays-Bas", "eux-mêmes", "porte-parole", "Notre-Dame", "puisqu'", "week-end", "quelqu'un", "celles-ci", "chef-lieu"])\
    .setPrefixPattern("\\A([^\\s\\p{L}\\d\\$\\.#]*)")\
    .setSuffixPattern("([^\\s\\p{L}\\d]?)([^\\s\\p{L}\\d]*)\\z")\
    .setInfixPatterns([
      "([\\p{L}\\w]+'{1})",
      "([\\$#]?\\d+(?:[^\\s\\d]{1}\\d+)*)",
      "((?:\\p{L}\\.)+)",
      "((?:\\p{L}+[^\\s\\p{L}]{1})+\\p{L}+)",
      "([\\p{L}\\w]+)"
    ])

posTagger = PerceptronApproach() \
    .setNIterations(1) \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("pos") \
    .setPosCol("tags")
    
pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector, 
    tokenizer,
    posTagger
])

In [7]:
# Let's train our Pipeline by using our training dataset
model = pipeline.fit(training_data)

This is our testing DataFrame where we get some sentences in French. We are going to use our trained Pipeline to transform these sentence and predict each token's `Part Of Speech`.

In [8]:
dfTest = spark.createDataFrame([
    "Je sens qu'entre ça et les films de médecins et scientifiques fous que nous avons déjà vus, nous pourrions emprunter un autre chemin pour l'origine.",
    "On pourra toujours parler à propos d'Averroès de décentrement du Sujet."
], StringType()).toDF("text")

In [9]:
predict = model.transform(dfTest)

In [10]:
predict.select("token.result", "pos.result").show()

Py4JJavaError: An error occurred while calling o337.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, localhost, executor driver): org.apache.spark.SparkException: 
Error from python worker:
  Traceback (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 184, in _run_module_as_main
      mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
    File "/usr/lib/python3.8/runpy.py", line 110, in _get_module_details
      __import__(pkg_name)
    File "<frozen importlib._bootstrap>", line 991, in _find_and_load
    File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
    File "<frozen importlib._bootstrap>", line 655, in _load_unlocked
    File "<frozen importlib._bootstrap>", line 618, in _load_backward_compatible
    File "<frozen zipimport>", line 259, in load_module
    File "/home/loan/venv/spark-nlp-2-4-4/lib/pyt